In [3]:
from dados.raw.utils.postgres_interactions import PostgresETL
import os
from dotenv import load_dotenv
import pandas as pd
import basedosdados as bd
from typing import Dict, Any

### Geração dos dados de produção rural

In [ ]:



load_dotenv()

## queries extracao vegetal
extracao_vegetal_censo_2006_query = """
select 
ano, 
nome_regiao_integracao,
produto,  
round(sum(quantidade_produzida),2) as quantidade_produzida,
round(sum(valor_producao),2) as valor_producao 
from pa_indexadores_producao_rural.extracao_vegetal_censo_2006
group by 1,2,3"""

extracao_vegetal_censo_2017_query = """
select 
ano, 
nome_regiao_integracao,
produto,
round(sum(quantidade_produzida),2) as quantidade_produzida,
round(sum(valor_producao),2) as valor_producao 
from pa_indexadores_producao_rural.extracao_vegetal_censo_2017
group by 1,2,3"""
extracao_vegetal_pevs_query = """
select 
ano, 
nome_regiao_integracao,
produto,
round(sum(quantidade_produzida),2) as quantidade_produzida,
round(sum(valor_producao),2) as valor_producao 
from pa_indexadores_producao_rural.extracao_vegetal_pevs
group by 1,2,3"""

## queries lavoura permanenten
lavoura_permanente_censo_2006_query = """
select 
ano, 
nome_regiao_integracao,
produto, 
round(sum(quantidade_produzida),2) as quantidade_produzida,
round(sum(valor_producao),2) as valor_producao 
from pa_indexadores_producao_rural.lavoura_permanente_censo_2006
group by 1,2,3"""

lavoura_permanente_censo_2017_query = """
select 
ano, 
nome_regiao_integracao,
produto, 
round(sum(quantidade_produzida),2) as quantidade_produzida,
round(sum(valor_producao),2) as valor_producao 
from pa_indexadores_producao_rural.lavoura_permanente_censo_2017
group by 1,2,3"""
lavoura_permanente_pam_query = """
select 
ano, 
nome_regiao_integracao,
produto, 
round(sum(quantidade_produzida),2) as quantidade_produzida,
round(sum(valor_producao),2) as valor_producao 
from pa_indexadores_producao_rural.lavoura_permanente_pam
group by 1,2,3"""


## queries lavoura temporaria
lavoura_temporaria_censo_2006_query = """
select 
ano, 
nome_regiao_integracao,
produto, 
round(sum(quantidade_produzida),2) as quantidade_produzida,
round(sum(valor_producao),2) as valor_producao 
from pa_indexadores_producao_rural.lavoura_temporaria_censo_2006
group by 1,2,3"""

lavoura_temporaria_censo_2017_query = """
select 
ano, 
nome_regiao_integracao,
produto, 
round(sum(quantidade_produzida),2) as quantidade_produzida,
round(sum(valor_producao),2) as valor_producao 
from pa_indexadores_producao_rural.lavoura_temporaria_censo_2017
group by 1,2,3"""

lavoura_temporaria_pam_query = """
select 
ano, 
nome_regiao_integracao,
produto, 
round(sum(quantidade_produzida),2) as quantidade_produzida,
round(sum(valor_producao),2) as valor_producao 
from pa_indexadores_producao_rural.lavoura_temporaria_pam
group by 1,2,3"""

def df_to_json(df: pd.DataFrame, coluna_valor: str) -> Dict[str, Any]:
    """
    Converte um DataFrame para uma estrutura aninhada em formato JSON com a seguinte hierarquia:
    tipo_agricultura -> nome_regiao_integracao -> produto -> ano -> valor
    
    Args:
        df (pd.DataFrame): DataFrame contendo os dados
        coluna_valor (str): Nome da coluna que contém os valores a serem armazenados
        
    Returns:
        Dict[str, Any]: Estrutura aninhada em formato dicionário
    """
    vetor = {}
    
    # Primeiro nível: tipo_agricultura
    for tipo_agricultura, grupo_tipo in df.groupby('tipo_agricultura'):
        if tipo_agricultura not in vetor:
            vetor[tipo_agricultura] = {}
        
        # Segundo nível: nome_regiao_integracao (subregião)
        for nome, grupo_nome in grupo_tipo.groupby('nome_regiao_integracao'):
            if nome not in vetor[tipo_agricultura]:
                vetor[tipo_agricultura][nome] = {}
            
            # Terceiro nível: produto
            for produto, grupo_produto in grupo_nome.groupby('produto'):
                if produto not in vetor[tipo_agricultura][nome]:
                    vetor[tipo_agricultura][nome][produto] = {}
                
                # Quarto nível: ano -> valor
                for _, row in grupo_produto.iterrows():
                    ano = row['ano']
                    valor = row[coluna_valor]
                    vetor[tipo_agricultura][nome][produto][ano] = valor
    
    return vetor

with PostgresETL(
        host='localhost', 
        database=os.getenv("DB_GOLD_ZONE"), 
        user=os.getenv("POSTGRES_USER"), 
        password=os.getenv("POSTGRES_PASSWORD"),
        schema='pa_indexadores_producao_rural') as db:
    
    # Download dos dados
    extracao_vegetal_censo_2006_query = db.download_data(extracao_vegetal_censo_2006_query)
    extracao_vegetal_censo_2017_query = db.download_data(extracao_vegetal_censo_2017_query)
    extracao_vegetal_pevs_query = db.download_data(extracao_vegetal_pevs_query)
    
    lavoura_permanente_censo_2006_query = db.download_data(lavoura_permanente_censo_2006_query)
    lavoura_permanente_censo_2017_query = db.download_data(lavoura_permanente_censo_2017_query)
    lavoura_permanente_pam_query = db.download_data(lavoura_permanente_pam_query)
    
    lavoura_temporaria_censo_2006_query = db.download_data(lavoura_temporaria_censo_2006_query)
    lavoura_temporaria_censo_2017_query = db.download_data(lavoura_temporaria_censo_2017_query)
    lavoura_temporaria_pam_query = db.download_data(lavoura_temporaria_pam_query)
    


2025-06-02 19:35:26,074 - PostgresETL - INFO - Connected to database gold_zone at localhost
2025-06-02 19:35:26,320 - PostgresETL - INFO - Downloaded 533 rows from database
2025-06-02 19:35:26,362 - PostgresETL - INFO - Downloaded 689 rows from database
2025-06-02 19:35:28,683 - PostgresETL - INFO - Downloaded 22230 rows from database
2025-06-02 19:35:28,745 - PostgresETL - INFO - Downloaded 858 rows from database
2025-06-02 19:35:28,788 - PostgresETL - INFO - Downloaded 13 rows from database
2025-06-02 19:35:29,025 - PostgresETL - INFO - Downloaded 9750 rows from database
2025-06-02 19:35:29,063 - PostgresETL - INFO - Downloaded 689 rows from database
2025-06-02 19:35:29,086 - PostgresETL - INFO - Downloaded 13 rows from database
2025-06-02 19:35:29,638 - PostgresETL - INFO - Downloaded 20400 rows from database
2025-06-02 19:35:29,645 - PostgresETL - INFO - Disconnected from database


In [5]:
import json
import os
from typing import Dict, Any
import pandas as pd
from decimal import Decimal

# Função para salvar JSON em arquivo
def decimal_default(obj):
    """
    Função auxiliar para converter objetos Decimal para float
    durante a serialização JSON.
    """
    if isinstance(obj, Decimal):
        return float(obj)
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

def save_json(data: Dict[str, Any], filename: str) -> None:
    """
    Salva um dicionário como arquivo JSON.
    
    Args:
        data (Dict[str, Any]): Dicionário a ser salvo
        filename (str): Nome do arquivo
    """
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4, default=decimal_default)
    print(f"Arquivo {filename} salvo com sucesso!")

# Já temos os vetores para PEVS e censo combinado conforme código fornecido
vetor_valor_pevs = df_to_json(extracao_vegetal_pevs_query, 'valor_producao')
vetor_quantidade_pevs = df_to_json(extracao_vegetal_pevs_query, 'quantidade_produzida')
censo_combinado = pd.concat([extracao_vegetal_censo_2006_query, extracao_vegetal_censo_2017_query], ignore_index=True)
vetor_valor_censo = df_to_json(censo_combinado, 'valor_producao')
vetor_quantidade_censo = df_to_json(censo_combinado, 'quantidade_produzida')

# Criar vetores para lavoura permanente (PAM)
lavoura_permanente_pam_vetores = {
    'valor': df_to_json(lavoura_permanente_pam_query, 'valor_producao'),
    'quantidade': df_to_json(lavoura_permanente_pam_query, 'quantidade_produzida')
}

# Criar vetores para lavoura temporária (PAM)
lavoura_temporaria_pam_vetores = {
    'valor': df_to_json(lavoura_temporaria_pam_query, 'valor_producao'),
    'quantidade': df_to_json(lavoura_temporaria_pam_query, 'quantidade_produzida')
}

# Criar vetores para lavoura permanente (censo combinado)
lavoura_permanente_censo_combinado = pd.concat([
    lavoura_permanente_censo_2006_query, 
    lavoura_permanente_censo_2017_query
], ignore_index=True)

lavoura_permanente_censo_vetores = {
    'valor': df_to_json(lavoura_permanente_censo_combinado, 'valor_producao'),
    'quantidade': df_to_json(lavoura_permanente_censo_combinado, 'quantidade_produzida')
}

# Criar vetores para lavoura temporária (censo combinado)
lavoura_temporaria_censo_combinado = pd.concat([
    lavoura_temporaria_censo_2006_query, 
    lavoura_temporaria_censo_2017_query
], ignore_index=True)

lavoura_temporaria_censo_vetores = {
    'valor': df_to_json(lavoura_temporaria_censo_combinado, 'valor_producao'),
    'quantidade': df_to_json(lavoura_temporaria_censo_combinado, 'quantidade_produzida')
}

# Criar diretório para os arquivos JSON se não existir
output_dir = 'json_output'
os.makedirs(output_dir, exist_ok=True)

# Salvar todos os JSONs
# PEVS
save_json(vetor_valor_pevs, f'{output_dir}/extracao_vegetal_pevs_valor.json')
save_json(vetor_quantidade_pevs, f'{output_dir}/extracao_vegetal_pevs_quantidade.json')

# Censo Extração Vegetal
save_json(vetor_valor_censo, f'{output_dir}/extracao_vegetal_censo_valor.json')
save_json(vetor_quantidade_censo, f'{output_dir}/extracao_vegetal_censo_quantidade.json')

# Lavoura Permanente PAM
save_json(lavoura_permanente_pam_vetores['valor'], f'{output_dir}/lavoura_permanente_pam_valor.json')
save_json(lavoura_permanente_pam_vetores['quantidade'], f'{output_dir}/lavoura_permanente_pam_quantidade.json')

# Lavoura Temporária PAM
save_json(lavoura_temporaria_pam_vetores['valor'], f'{output_dir}/lavoura_temporaria_pam_valor.json')
save_json(lavoura_temporaria_pam_vetores['quantidade'], f'{output_dir}/lavoura_temporaria_pam_quantidade.json')

# Lavoura Permanente Censo
save_json(lavoura_permanente_censo_vetores['valor'], f'{output_dir}/lavoura_permanente_censo_valor.json')
save_json(lavoura_permanente_censo_vetores['quantidade'], f'{output_dir}/lavoura_permanente_censo_quantidade.json')

# Lavoura Temporária Censo
save_json(lavoura_temporaria_censo_vetores['valor'], f'{output_dir}/lavoura_temporaria_censo_valor.json')
save_json(lavoura_temporaria_censo_vetores['quantidade'], f'{output_dir}/lavoura_temporaria_censo_quantidade.json')

print("Todos os arquivos JSON foram gerados com sucesso!")

KeyError: 'tipo_agricultura'

In [6]:
vetor_quantidade_censo

NameError: name 'vetor_quantidade_censo' is not defined

### Explorando problemas de compatibilização de produtos

In [25]:
import pandas as pd

path = os.path.join(os.getcwd(), 'temp')


path_tbs_extensa = [file for file in os.listdir(path) if file.startswith('Tb')]

tabela_produto_dict = {}

for tabela in path_tbs_extensa:
    tabela_path = os.path.join(path, tabela)
    try:
        df = pd.read_excel(tabela_path)
        if 'Produto' in df.columns:
            produtos_unicos = df['Produto'].dropna().unique().tolist()
            tabela_produto_dict[tabela] = produtos_unicos
        else:
            print(f"A tabela {tabela} não possui a coluna 'Produto'.")
    except Exception as e:
        print(f"Erro ao processar a tabela {tabela}: {e}")


produtos = list(set(item for sublist in tabela_produto_dict.values() for item in sublist))
produtos


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


['AndirobaOleo',
 'CestinhaCarocinho',
 'CajuAçuFruto',
 'TaperebaFruto',
 'Peneira',
 'Alface',
 'BacuriFruto',
 'Portal',
 'CestinhaHamburger',
 'Cama',
 'CascaMarapuana',
 'Almofada',
 'CastanhaDoPara',
 'CascaAndiroba',
 'Amendoim',
 'SementeDeCumaru',
 'Estorac',
 'OleoPracaxi',
 'Frango',
 'SementeCumaru',
 'VassouraArtesanal',
 'PaneiroPequeno',
 'LeiteAnani',
 'Piso',
 'Comodas',
 'BacabaCaroco',
 'CascaCarapanauba',
 'FrangoCaipira',
 'TucumaFruto',
 'Paneiro',
 'CascaBarbatimao',
 'Pupunha',
 'Mel',
 'Fava',
 'Abóbora',
 'CestinhaDedal',
 'LeiteJatoba',
 'CarneBovina',
 'Compensado',
 'PimentaDoReino',
 'PimeitaDoReino',
 'Quiabo',
 'PeneiraCorujinha',
 'FavaDeJuca',
 'CascaPariri',
 'CheiroVerde',
 'Degrau',
 'Breu',
 'Farinha',
 'Peixe',
 'Abacaxi',
 'VassouraRegional',
 'Paxiuba',
 'LeiteSucuuba',
 'Guarumã',
 'AcaiCaroco',
 'VassouraInaja',
 'CaixaPia',
 'Soja',
 'Bau3',
 'CurauaFibra',
 'MadeiraVermelha',
 'Caimbe',
 'EscadaDeJabuti',
 'Lenha',
 'PaleteMadeira',
 'Pripri

In [19]:
import os
import pandas as pd

path = os.path.join(os.getcwd(), 'temp')
path_tbs_extensa = [file for file in os.listdir(path) if file.startswith('Tb')]

tabela_produto_dict = {}

soma_global = {}

for tabela in path_tbs_extensa:
    tabela_path = os.path.join(path, tabela)
    try:
        df = pd.read_excel(tabela_path)

        if all(col in df.columns for col in ['Produto', 'Unidade', 'Quantidade']):
            df_filtrado = df[['Produto', 'Unidade', 'Quantidade']].dropna()
            df_filtrado['Unidade'] = df_filtrado['Unidade'].str.lower()
            df_filtrado['Quantidade'] = pd.to_numeric(df_filtrado['Quantidade'], errors='coerce').fillna(0)

            agrupado = (
                df_filtrado
                .groupby(['Produto', 'Unidade'], dropna=True)['Quantidade']
                .sum()
                .reset_index()
            )

            tabela_dict = {}

            for _, row in agrupado.iterrows():
                produto = row['Produto']
                unidade = row['Unidade']
                quantidade = row['Quantidade']

                if produto not in tabela_dict:
                    tabela_dict[produto] = {}
                tabela_dict[produto][unidade] = quantidade

                if produto not in soma_global:
                    soma_global[produto] = {}
                if unidade not in soma_global[produto]:
                    soma_global[produto][unidade] = 0
                soma_global[produto][unidade] += quantidade

            tabela_produto_dict[tabela] = tabela_dict

        else:
            print(f"A tabela {tabela} não possui todas as colunas necessárias.")

    except Exception as e:
        print(f"Erro ao processar a tabela {tabela}: {e}")

linhas = []

for produto, unidades in soma_global.items():
    for unidade, quantidade in unidades.items():
        linhas.append({
            'produto': produto,
            'unidade': unidade,
            'soma_quantidade': quantidade
        })

df_resultado = pd.DataFrame(linhas)



WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


### Algoritmo de indexação do trabalho

In [7]:
lavoura_permanente_pam_query 
lavoura_permanente_censo_2017_query 
lavoura_permanente_censo_2006_query


,ano,nome_regiao_integracao,produto,quantidade_produzida,valor_producao
0,2006,Tapajós,pupunha-cacho,1.00,1.80
1,2006,Lago de Tucuruí,pitanga,0.00,0.00
2,2006,Carajás,lichia,0.00,0.00
3,2006,Carajás,jabuticaba,0.00,0.00
4,2006,Xingu,guaraná,0.00,0.11
...,...,...,...,...,...
853,2006,Carajás,nectarina,0.00,0.00
854,2006,Lago de Tucuruí,pera,0.00,0.00
855,2006,NaN,romã,0.00,0.00
856,2006,Rio Capim,nêspera,0.00,0.00
